# Stack echelle spectra

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from astropy.io import fits

sys.path.insert(0, '../')

In [2]:
from nephelion import RawImage, ImageSet

Find all of the spectra: 

In [3]:
from glob import glob

home = os.path.expanduser('~')
target_1 = 'EPIC211928486'
target_2 = 'EPIC211966629'

prog_nights = ['Q4UW09/UT171128', 'Q4UW09/UT171204', 'Q4UW06/UT171208', 
               'Q4UW06/UT171211', 'Q4UW06/UT171211']

target_1_paths = []
target_2_paths = []

for prog_night in prog_nights:
    target_1_paths.append(glob('{0}/data/{1}/{2}.????.fits'
                               .format(home, prog_night, target_1, )))
    target_2_paths.append(glob('{0}/data/{1}/{2}.????.fits'
                               .format(home, prog_night, target_2, )))

Concatenate lists of spectra from each target, toss any empty lists: 

In [4]:
all_paths = target_1_paths + target_2_paths
all_paths = [p for p in all_paths if len(p) > 0]

For all spectra of one target on one night: 
* remove cosmic rays from the individual spectra
* stack cleaned spectra
* save to a FITS file with updated header

In [8]:
for paths in all_paths:
    images = ImageSet([RawImage.from_fits(path) for path in paths])
    image, header, name = images.clean_and_stack(sigclip=2)
    
    name = name.replace(' ', '')
    date = header['DATE-OBS'].split('T')[0]
    stacked_path = '../data/stack_{0}_{1}.fits'.format(name, date)
    fits.writeto(stacked_path, image, header, overwrite=True)